In [1]:
import os
from tqdm.notebook import tqdm
import numpy as np

import calibpy.calibration as calib

SCREEN_WH = (1920, 1080)
FRAME_RATE = 20

In [2]:
# Split video into image frames (minimum of 3 if later intrinsic estimation)
notebook_path = os.path.dirname(os.path.realpath('__file__'))
video_path = os.path.join(notebook_path, '..', 'res/checkerboard.mp4')
frames = calib.convert_video_to_frames(video_path, frame_rate=int((1/FRAME_RATE)*1000), save_imgs=False)
print(len(frames))

179


In [3]:
# Select initial tracking points (minimum of 4 if later DLT) and track them across frames
points_frames = calib.track_points_seq(frames, SCREEN_WH)
num_frames = points_frames.shape[0]

In [4]:
# Estimate homographies across frames
board_points = calib.generate_homogeneous_board_points()
H_frames = np.zeros((num_frames, 3, 3))
for idx in range(num_frames):
    H_frames[idx] = calib.estimate_homography(points_frames[idx], board_points)

In [5]:
# Estimate intrinsic camera matrix from homography sequence
K = calib.estimate_camera_intrinsic_matrix(H_frames)

LinAlgError: Matrix is not positive definite